In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['mnist_train.csv', 'mnist_test.csv']


In [27]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [28]:
train = pd.read_csv('../input/mnist_train.csv')
test = pd.read_csv('../input/mnist_test.csv')

In [29]:
train = np.array(train)
test = np.array(test)

In [30]:
train_X = train[:,1:]
train_Y = pd.get_dummies(train[:,0])
print(train_X.shape)
test_X = test[:,1:]
test_Y = pd.get_dummies(test[:,0])
print(test_Y.shape)

(60000, 784)
(10000, 10)


In [31]:
train_X = train_X.reshape(-1, 28, 28)
test_X = test_X.reshape(-1, 28, 28)

In [32]:
train_X.shape

(60000, 28, 28)

In [33]:
tf.reset_default_graph()

In [35]:
# Training Parameters
learning_rate = 0.001
training_iters = 250
batch_size = 128

In [38]:
# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 64 # hidden layer num of features: n_neurons
num_classes = 10 # MNIST total classes (0-9 digits)

In [39]:
# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [40]:
def RNN(x):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)
    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)
    # Define a lstm cell with tensorflow
    RNN_cell = tf.nn.rnn_cell.BasicRNNCell(num_hidden)

    # Get lstm cell output
    outputs, states = tf.nn.static_rnn(RNN_cell, x, dtype=tf.float32)

#     out = tf.matmul(outputs[-1], weights['out']) + biases['out']

    out = tf.layers.dense(states, num_classes)

    # Softmax activation, using rnn inner loop last output
    
    out = tf.nn.softmax(out)
    
    return out

In [41]:
prediction = RNN(X)

In [42]:
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [43]:
# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [44]:
sess = tf.Session()
sess.run(init) 

for i in range(training_iters):
    for batch in range(len(train_X)//batch_size):
        batch_x = train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
        batch_y = train_Y[batch*batch_size:min((batch+1)*batch_size,len(train_Y))]    

        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
            # Calculate batch loss and accuracy
        loss = sess.run([loss_op], feed_dict={X: batch_x, Y: batch_y})
            
#     acc = sess.run([accuracy], feed_dict={X: batch_x, Y: batch_y})
    predTest = sess.run(prediction , feed_dict={X:test_X})

    p = np.argmax(predTest,1)
    t = np.argmax(np.array(test_Y),1)

    acc = accuracy_score(p,t)
    print("Iter "+str(i)+" Out of",training_iters , " Loss= ",loss, "acc=",acc )
print("Optimization Finished!")
        
#     print("Step " + str(i) + ",        Batch Loss= ",loss, ",       Training Accuracy= ",acc)
    
# print("Optimization Finished!")


Iter 0 Out of 250  Loss=  [1.9928792] acc= 0.461
Iter 1 Out of 250  Loss=  [1.981642] acc= 0.4859
Iter 2 Out of 250  Loss=  [1.9901857] acc= 0.491
Iter 3 Out of 250  Loss=  [1.9934132] acc= 0.491
Iter 4 Out of 250  Loss=  [1.9811851] acc= 0.4889
Iter 5 Out of 250  Loss=  [1.9731381] acc= 0.4902
Iter 6 Out of 250  Loss=  [1.9979453] acc= 0.4924
Iter 7 Out of 250  Loss=  [1.9877051] acc= 0.4954
Iter 8 Out of 250  Loss=  [1.9913046] acc= 0.4927
Iter 9 Out of 250  Loss=  [1.9857116] acc= 0.4957
Iter 10 Out of 250  Loss=  [1.9904785] acc= 0.4961
Iter 11 Out of 250  Loss=  [1.9820148] acc= 0.4956
Iter 12 Out of 250  Loss=  [1.9951327] acc= 0.4936
Iter 13 Out of 250  Loss=  [1.9705058] acc= 0.5005
Iter 14 Out of 250  Loss=  [1.9690747] acc= 0.5015
Iter 15 Out of 250  Loss=  [1.9724803] acc= 0.4994
Iter 16 Out of 250  Loss=  [1.9747958] acc= 0.5014
Iter 17 Out of 250  Loss=  [1.9993916] acc= 0.5093
Iter 18 Out of 250  Loss=  [1.9533737] acc= 0.5137
Iter 19 Out of 250  Loss=  [1.9631596] acc= 0

Iter 161 Out of 250  Loss=  [1.9283694] acc= 0.5303
Iter 162 Out of 250  Loss=  [1.926333] acc= 0.5326
Iter 163 Out of 250  Loss=  [1.9514424] acc= 0.5336
Iter 164 Out of 250  Loss=  [1.9079742] acc= 0.5286
Iter 165 Out of 250  Loss=  [1.9508761] acc= 0.5306
Iter 166 Out of 250  Loss=  [1.9345518] acc= 0.5344
Iter 167 Out of 250  Loss=  [1.9193432] acc= 0.5324
Iter 168 Out of 250  Loss=  [1.963038] acc= 0.535
Iter 169 Out of 250  Loss=  [1.9129093] acc= 0.5331
Iter 170 Out of 250  Loss=  [1.9630983] acc= 0.5369
Iter 171 Out of 250  Loss=  [1.934934] acc= 0.5346
Iter 172 Out of 250  Loss=  [1.9481406] acc= 0.5374
Iter 173 Out of 250  Loss=  [1.9435656] acc= 0.5331
Iter 174 Out of 250  Loss=  [1.9309247] acc= 0.536
Iter 175 Out of 250  Loss=  [1.9543765] acc= 0.5327
Iter 176 Out of 250  Loss=  [1.9454422] acc= 0.5333
Iter 177 Out of 250  Loss=  [1.9357231] acc= 0.5309
Iter 178 Out of 250  Loss=  [1.9645045] acc= 0.5376
Iter 179 Out of 250  Loss=  [1.9621534] acc= 0.5345
Iter 180 Out of 2

In [45]:
while(True):
    r = np.random.randint(9000)
    test_img = np.reshape(test_X[r], (28,28))
    plt.imshow(test_img, cmap="gray")
    test_pred = sess.run(prediction, feed_dict = {X:[test_X[r]]})
    print("Model : I think it is :    ",np.argmax(test_pred))
    plt.show()
    
    if input("Enter n to exit")=='n':
        break
clear_output();

In [46]:
wrong = test_X[t!=p]
wrong.shape

(4734, 28, 28)

In [47]:
a,b,c = wrong.shape

In [48]:
while(True):
    r=np.random.randint(a)
    plt.imshow(wrong[r].reshape((28,28)),cmap="gray")
    test_pred_1=sess.run(prediction, feed_dict = {X:[wrong[r]]})
    print("Model : I think it is :    ",np.argmax(test_pred_1))
    plt.show()
    if input("Enter n to exit")=='n':
        break
clear_output();

In [20]:
p = np.argmax(predTest,1)
print(p)
t = np.argmax(np.array(test_Y),1)
print(t)
acc = accuracy_score(p,t)
print(acc*100)

[7 0 1 ... 9 0 6]
[7 2 1 ... 4 5 6]
53.43


In [49]:
print("Saving Weights")
saver = tf.train.Saver()
saver.save(sess,"weights_"+str(i)+"/weights.ckpt")
print("Weights Saved")

Saving Weights
Weights Saved
